In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import torchreid
from scipy.spatial.distance import cosine
from deep_sort_realtime.deepsort_tracker import DeepSort
from sklearn.metrics.pairwise import cosine_similarity
import torch
import time
import numpy
import matplotlib.pyplot as plt
#time
prev_time = 0

# Initializing the models
yolo_model = YOLO("yolo11n.pt")
extractor = torchreid.utils.FeatureExtractor(
    model_name='osnet_x1_0',
    device='cpu'
)

# Initializing the DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=3)

# Gallery to store unique person embeddings
gallery = {}
next_person_id = 0
person_id_map = {}

#for preset images 
preset_faces = {
    "Nishan": "C:/Users/nisha/OneDrive - Indian Institute of Technology Guwahati/Documents/projects/CCTV/data/nishan.png",
    
}

for name, path in preset_faces.items():
    img = cv2.imread(path)
    if img is None:
        print(f"Could not load image: {path}")
        continue

    resized = cv2.resize(img, (128, 256))
    feature = extractor([resized])[0].cpu()

    gallery[name] = {
        'embedding': feature,
        'face': resized,
        'is_good_enough': True  # Prevent overwriting
    }

    print(f"Added {name} to gallery")


#defining a function whether the current image is better than the previously stored images
def is_better_image(new_img, old_img):
    def sharpness(img):
        return cv2.Laplacian(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.CV_64F).var()

    def area(img):
        return img.shape[0] * img.shape[1]

    return sharpness(new_img) + area(new_img) > sharpness(old_img) + area(old_img)

#defining function if the already stored image is good enough to be kept or needs to be replaced

def is_good_enough(image, sharp_thresh=100.0, brightness_thresh=(60, 200)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Sharpness using variance of Laplacian
    sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()

    # Brightness
    mean_brightness = np.mean(gray)

    return (
        sharpness > sharp_thresh and
        brightness_thresh[0] < mean_brightness < brightness_thresh[1]
    )



cap = cv2.VideoCapture(0)
min_conf = 0.5
while True:
    detections = []
    check, frame = cap.read()

    #getting the fps
    cur_time = time.time()
    fps = 1/(cur_time - prev_time)
    fps = round(fps*100)/100
    prev_time = cur_time

    if not check:
        break
    
    # YOLO detection
    result = yolo_model(frame, verbose=False)[0]
    for box in result.boxes:
        if box.cls[0] != 0 or box.conf[0] < min_conf:
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    # DeepSORT tracking
    tracks = tracker.update_tracks(detections, frame=frame)
    
    for track in tracks:
        if not track.is_confirmed():
            continue
        
        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        
        # Check if the track ID is in our person_id map
        if track_id not in person_id_map:
            # New track ID, perform ReID check
            cropped = frame[t:b, l:r]
            if cropped.size == 0:
                continue
            
            resized = cv2.resize(cropped, (128, 256))
            feature = extractor([resized])[0].cpu()

            best_match_id = None
            best_similarity_score = 0.0

            # Compare with gallery
            if gallery:
                gallery_embeddings = np.array([data['embedding'].numpy() for data in gallery.values()])
                similarity_scores = cosine_similarity(
                    feature.unsqueeze(0).numpy(),
                    gallery_embeddings
                )[0]
                
                max_score_idx = np.argmax(similarity_scores)
                best_similarity_score = similarity_scores[max_score_idx]
                
                # Minimum similarity threshold
                if best_similarity_score > 0.6:
                    best_match_id = list(gallery.keys())[max_score_idx]

            if best_match_id is not None:
                person_id_map[track_id] = best_match_id
            else:
                current_person_id = next_person_id
                next_person_id += 1

                if is_good_enough(resized):
                    gallery[current_person_id] = {
                        'embedding': feature,
                        'face': resized,
                        'locked': True  # lock to prevent future updates
                    }
                    print(f"Stored GOOD face for new person {current_person_id}")
                else:
                    gallery[current_person_id] = {
                        'embedding': feature,
                        'face': resized,
                        'locked': False  # allow future updates
                    }
                    print(f"Stored TEMP face for new person {current_person_id}")

                person_id_map[track_id] = current_person_id
        
        # Draw
        pid = person_id_map.get(track_id, "Unknown")
        label = f"{pid}" if isinstance(pid, str) and not pid.isdigit() else f"Person {pid}"

        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, label, (l, t - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)


    cv2.putText(frame,str(fps),(20,50),cv2.FONT_HERSHEY_PLAIN,1.5 ,(0,100,250),2)
    cv2.imshow("ReID + Tracking", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import matplotlib.pyplot as plt
import cv2

saved_faces = [data['face'] for data in gallery.values()]
n = len(saved_faces)

if n == 0:
    print("No faces saved yet.")
else:
    # Grid dimensions
    cols = min(n, 5)  # limit to 5 columns
    rows = (n + cols - 1) // cols

    plt.figure(figsize=(3 * cols, 4 * rows))

    for i, face in enumerate(saved_faces):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
        plt.title(f"Person {list(gallery.keys())[i]}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()
